In [1]:
import numpy as np, matplotlib.pyplot as plt
from scipy import fft

from tqdm import tqdm

import time
from glob import glob
from natsort import natsorted

import sys, os
current_path = os.path.abspath('')
sys.path.insert(0, os.path.abspath(os.path.join(current_path, '..')))

from activemodelbplus.integrator import Model, Stencil, Integrator

In [ ]:
# Parameters for normal equilibrium spinodal decomposition.
dx, dy = 1, 1
stencil = Stencil(dt=1e-2, dx=dx, dy=dy)
model = Model(a=-0.25, b=0, c=0.25, kappa=1, lamb=0, zeta=0, T=0)
phi0 = 0
variance = 1.

Nx, Ny = 512, 512
initial = np.random.normal(loc=phi0, scale=np.sqrt(variance), size=(Ny, Nx))
initial += phi0 - np.average(initial)

sim = Integrator(initial, stencil, model)

def trajectory(s):
    trajectory.time += [s.time]
    trajectory.field += [s.field.copy()]
trajectory.time = []
trajectory.field = []

tsample = np.geomspace(1e0, 1e6, 7)
sim.run_for_time(tsample[-1], show_progress=True, max_updates=1000,
                 t_interrupt=tsample, f_interrupt=trajectory)
assert np.all(np.isclose(tsample, trajectory.time))

params = model.as_dict()
params.update(stencil.as_dict())
params
path = f'trajectory_{int(time.time())}.npz'
np.savez(path, time=trajectory.time, field=trajectory.field, **params)

Checking correspondence between discrete and continuous Fourier transforms by considering the humble Gaussian:

In [ ]:
N = 1000
L = 200
z = np.linspace(0, L, N)
dz = z[1] - z[0]
sigma = 2
mu = 50
fz = np.exp(-0.5 * (z-mu)**2 / sigma**2) / (2*np.pi * sigma**2)**0.5
assert np.isclose(np.trapezoid(fz, z), 1)

k = 2*np.pi * fft.fftshift(fft.fftfreq(N, d=dz))
fk = fft.fftshift(fft.fft(fz)) * dz
fk_exact = np.exp(-0.5 * (sigma*k)**2)
assert np.allclose(np.abs(fk), fk_exact)

fig, (ax1, ax2) = plt.subplots(nrows=2)
ax1.plot(z, fz)
ax1.set_xlabel(r'$x$')
ax1.set_ylabel(r'$f(x)$')
ax2.plot(k, np.abs(fk))
ax2.plot(k, fk_exact)
ax2.set_xlabel(r'$k$')
ax2.set_ylabel(r'$\widetilde{f}(k)$')

plt.show()

Test Fourier transform of the field. It should show a ring around the centre due to rotational symmetry.

In [ ]:
path = natsorted(glob('trajectory*.npz'))[-1]
traj = np.load(path)
phi = traj['field'][-1]
dx = traj['dx']
dy = traj['dy']
Ny, Nx = traj['field'][0].shape
phi0 = np.average(traj['field'][0])

phiq = fft.fftshift(fft.fft2(phi-phi0)) * dx * dy
kx = 2*np.pi * fft.fftshift(fft.fftfreq(Nx, d=dx))
ky = 2*np.pi * fft.fftshift(fft.fftfreq(Ny, d=dy))
Kx, Ky = np.meshgrid(kx, ky, indexing='xy')

plt.pcolormesh(Kx, Ky, np.abs(phiq))
plt.gca().set_aspect(1)
plt.xlabel('$k_x$')
plt.ylabel('$k_y$')
plt.show()

Calculate circularly averaged $S(q)$:

In [ ]:
bin_edges = np.linspace(0, 3, 300)[1:]
k = 0.5 * (bin_edges[1:] + bin_edges[:-1])

path = natsorted(glob('trajectory*.npz'))[-1]
traj = np.load(path)

dx = traj['dx']
dy = traj['dy']
Ny, Nx = traj['field'][0].shape
Lx = Nx * dx
Ly = Ny * dy
A = Lx * Ly
phi0 = np.average(traj['field'][0])

kx = 2*np.pi * fft.fftshift(fft.fftfreq(Nx, d=dx))
ky = 2*np.pi * fft.fftshift(fft.fftfreq(Ny, d=dy))
Kx, Ky = np.meshgrid(kx, ky, indexing='xy')
K = (Kx**2 + Ky**2)**0.5
indices = np.digitize(K, bin_edges, right=True)

print('------------')
print('  t    <R>')
print('------------')

for t, phi in zip(traj['time'], traj['field']):
    assert np.isclose(np.average(phi), phi0)
    phiq = fft.fftshift(fft.fft2(phi-phi0)) * dx * dy
    phiq = np.abs(phiq)**2

    S = np.zeros_like(k)
    for l in range(S.size):
        S[l] = np.average(phiq[indices == l])

    from scipy.integrate import simpson
    k_avg = simpson(k * S, k) / simpson(S, k)
    print(fr' 10^{np.log10(t):.0g} {2*np.pi / k_avg:>5.1f}')

    plt.plot(k / k_avg, S * k_avg**2 / A, label=rf'$10^{np.log10(t):.0g}$')
print('------------')

guide = lambda k: 1/k**3
k = np.geomspace(0.5, 5)
pl, = plt.plot(k, guide(k), '--', c='grey')
kref = 2
plt.text(0.8*kref, guide(kref), r'$k^{-3}$', fontsize=8, ha='right', va='center', c=pl.get_color())

guide = lambda k: 2.5e-2/k
k = np.geomspace(1, 30)
pl, = plt.plot(k, guide(k), ':', c='brown')
kref = 10
plt.text(kref, 0.5*guide(kref), r'$k^{-1}$', fontsize=8, ha='center', va='center', c=pl.get_color())

plt.legend(loc='best', title=r'$t=$')
plt.xlabel(r'$k / \langle k \rangle$')
plt.ylabel(r'$S(k) \langle k \rangle^2 / A$')
plt.gca().set_xscale('log')
plt.gca().set_yscale('log')

plt.show()